# Optimization Of Facility Location In Switzerland

Below is an optimization model that finds the best location to place an Amazon warehouse in Switzerland. Details about the assumptions taken when building the model are discussed in the report.

In [18]:
!pip install pandas
!pip install numpy
!pip install ortools
!pip install geopy

## Installation of required libraries

In [19]:
import pandas as pd
import numpy as np
import time
import math
from ortools.sat.python import cp_model as cp

## Data loading

In [20]:
#Distances between 17 of the largest cities in Switzerland
dist = pd.read_csv('dist.csv')

#Population of cities in Switzerland
population = pd.read_csv('csvData.csv')

## Data preprocessing

In [21]:
dist

,Node 1,Node 2,Distance (KM)
0,Zurich,Geneva,282
1,Zurich,Basel,85
2,Zurich,Bern,130
3,Zurich,Lausanne,225
4,Zurich,Winterthur,27
...,...,...,...
267,Montreux,Fribourg,60
268,Montreux,Chur,330
269,Montreux,Sion,68
270,Montreux,Lugano,278


In [22]:
#Transform the dist list to a dist-matrix
dist = dist.pivot(index='Node 1', columns='Node 2', values='Distance (KM)').fillna(0)
dist

Node 2,Basel,Bern,Biel,Chur,Fribourg,Geneva,Lausanne,Lucerne,Lugano,Montreux,Saint Gallen,Schaffhausen,Sion,Thun,Winterthur,Zug,Zurich
Node 1,,,,,,,,,,,,,,,,,
Basel,0,111,90,204,130,251,187,95,266,182,164,130,249,122,106,108,85
Bern,111,0,53,248,37,161,105,116,293,90,217,175,156,27,159,138,130
Biel,90,53,0,237,80,161,109,105,274,132,198,163,200,72,140,129,117
Chur,204,248,237,0,277,399,343,143,145,330,106,185,396,270,159,117,121
Fribourg,130,37,80,277,0,140,72,145,312,60,237,203,130,69,183,167,159
Geneva,251,161,161,399,140,0,71,266,370,95,360,325,163,190,301,290,282
Lausanne,187,105,109,343,72,71,0,209,380,29,304,269,95,137,246,233,225
Lucerne,95,116,105,143,145,266,209,0,171,196,143,101,267,97,77,31,51
Lugano,266,293,274,145,312,370,380,171,0,278,249,256,217,248,240,172,208


In [23]:
#Create a twodimensional array to later be used in modelling
distance = np.array(dist.astype(int))
distance

array([[  0, 111,  90, 204, 130, 251, 187,  95, 266, 182, 164, 130, 249,
        122, 106, 108,  85],
       [111,   0,  53, 248,  37, 161, 105, 116, 293,  90, 217, 175, 156,
         27, 159, 138, 130],
       [ 90,  53,   0, 237,  80, 161, 109, 105, 274, 132, 198, 163, 200,
         72, 140, 129, 117],
       [204, 248, 237,   0, 277, 399, 343, 143, 145, 330, 106, 185, 396,
        270, 159, 117, 121],
       [130,  37,  80, 277,   0, 140,  72, 145, 312,  60, 237, 203, 130,
         69, 183, 167, 159],
       [251, 161, 161, 399, 140,   0,  71, 266, 370,  95, 360, 325, 163,
        190, 301, 290, 282],
       [187, 105, 109, 343,  72,  71,   0, 209, 380,  29, 304, 269,  95,
        137, 246, 233, 225],
       [ 95, 116, 105, 143, 145, 266, 209,   0, 171, 196, 143, 101, 267,
         97,  77,  31,  51],
       [266, 293, 274, 145, 312, 370, 380, 171,   0, 278, 249, 256, 217,
        248, 240, 172, 208],
       [182,  90, 132, 330,  60,  95,  29, 196, 278,   0, 290, 256,  68,
        1

In [24]:
#Extract the list of cities in the dist matrix
cities = pd.Series(dist.index).str.replace('\xa0','')
cities

0            Basel
1             Bern
2             Biel
3             Chur
4         Fribourg
5           Geneva
6         Lausanne
7          Lucerne
8           Lugano
9         Montreux
10    Saint Gallen
11    Schaffhausen
12            Sion
13            Thun
14      Winterthur
15             Zug
16          Zurich
Name: Node 1, dtype: object

By comparing the list of cities displayed above with the list of cities displayed in population-dataframe below, we see that population-dataframe contains a lot of redundant cities.

In [25]:
population

,2022,name
0,341730,Zurich
1,183981,Geneve
2,164488,Basel
3,121631,Bern
4,116751,Lausanne
...,...,...
338,5061,Estavayer-le-Lac
339,5045,Sargans
340,5032,Greifensee
341,5031,Wallisellen / Wallisellen-Ost


In [26]:
#Remove the redundant cities
relevant_pop = population[population['name'].isin(cities)]
relevant_pop

,2022,name
0,341730,Zurich
2,164488,Basel
3,121631,Bern
4,116751,Lausanne
5,91908,Winterthur
7,63000,Lugano
13,42136,Thun
18,33863,Schaffhausen
20,32827,Fribourg
21,32429,Chur


Comparing the table above with the cities list, Geneve, Luzern and Biel are missing. This is because they are named differently in the two lists. We thus have to add them manually. 
Also, Sion is not in the population list. It is added manually as well based on the information found at https://all-populations.com/en/ch/population-of-sion.html.


In [27]:
diff_spelled_cities = ['Geneve', 'Luzern', 'Sankt Gallen', 'Biel/Bienne']
sion = {'2022': 32797, 'name':'Sion'}

#Add cities spelled differently in the two tables
relevant_pop = pd.concat((relevant_pop, population.loc[population['name'].isin(diff_spelled_cities)]))

#Add the population of Sion
relevant_pop = pd.concat((relevant_pop, pd.DataFrame([sion])))

#Sort the values in alphabetical order
relevant_pop = relevant_pop.sort_values('name').reset_index(drop=True)
relevant_pop

,2022,name
0,164488,Basel
1,121631,Bern
2,48614,Biel/Bienne
3,32429,Chur
4,32827,Fribourg
5,183981,Geneve
6,116751,Lausanne
7,63000,Lugano
8,57066,Luzern
9,22897,Montreux


In [28]:
relevant_pop_list = np.array(relevant_pop['2022'])
relevant_pop_list

array([164488, 121631,  48614,  32429,  32827, 183981, 116751,  63000,
        57066,  22897,  70572,  33863,  32797,  42136,  91908,  23435,
       341730])

## Optimization Model

In [33]:
warehouse_sizes = [100000,250000] #sqfeet
warehouse_costs = [10000000,25000000] #dollar/euro
# https://www.bizjournals.com/triad/news/2018/05/31/application-reveals-construction-costs-for.html
# https://constructionphysics.substack.com/p/construction-costs-around-the-world?s=r

sq_per_person = 1 #https://www.bigrentz.com/blog/amazon-warehouses-locations
share = 0.2 #market share amazon assumption over time
sq_per_person *= share

turnover = 40 #inventory turnover for a year
horizon = 15 #years
truck_cap = 400 #sqfeet # https://oceanair.net/full-truckload-shipping-are-you-maximizing-your-trailer-space/
km_cost = int(5*1.6)#https://www.cassinfo.com/freight-audit-payment/cass-transportation-indexes/march-2022 

truckspace_per_person = (turnover*horizon*sq_per_person)

transport_cost = int(truckspace_per_person / truck_cap * km_cost) #per km per feet of capacity
relevant_pop_list

array([164488, 121631,  48614,  32429,  32827, 183981, 116751,  63000,
        57066,  22897,  70572,  33863,  32797,  42136,  91908,  23435,
       341730])

In [30]:
before = time.time()

model = cp.CpModel()

# data
num_cities = len(cities)
    
# declare variables
x1 = []
x2 = []
c = []
for i in range(num_cities):
    x1.append(model.NewBoolVar("x1[%i]" % i))
    x2.append(model.NewBoolVar("x2[%i]" % i))
    c.append([])
    for j in range(num_cities):
        c[i].append(model.NewIntVar(0, warehouse_sizes[-1], "c[%i][%i]" % (i, j)))

z = model.NewIntVar(0, 999999999999, "z")
p = model.NewIntVar(0, 999999999999, "p")

# objective to minimize
# cost of giving capacity over a distance
model.Add(z == (sum([c[i][j] * distance[i][j] * transport_cost for i in range(num_cities) for j in range(num_cities)])))

# cost of warehouses
model.Add(p == sum([x1[i] * warehouse_costs[0] for i in range(num_cities)]) + sum([x2[i] * warehouse_costs[1] for i in range(num_cities)]))

# constraints
for j in range(num_cities):
    model.Add(sum([c[i][j] for i in range(num_cities)]) >= int(relevant_pop_list[j] * sq_per_person))
    
for i in range(num_cities):
    model.Add(sum([c[i][j] for j in range(num_cities)]) <= x1[i] * warehouse_sizes[0] + x2[i] * warehouse_sizes[1])
    
for i in range(num_cities):
    model.Add(x1[i] + x2[i] <= 1)
    
for i in range(num_cities):
    model.Add(sum([c[i][j] for j in range(num_cities)]) >= 0)

model.Minimize(z + p)

# solution and search
solver = cp.CpSolver()
status = solver.Solve(model)

if status == cp.OPTIMAL:
    print("wcost:", solver.Value(p))
    print("tcost:", solver.Value(z))
    print("cost:", solver.Value(z) + solver.Value(p))
    print("x1:", [solver.Value(x1[i]) for i in range(num_cities)])
    print("x2:", [solver.Value(x2[i]) for i in range(num_cities)])
    print("c:", [solver.Value(c[i][j]) for i in range(num_cities) for j in range(num_cities)])

print("NumConflicts:", solver.NumConflicts())
print("NumBranches:", solver.NumBranches())
after = time.time()
print(after - before)

wcost: 30000000
tcost: 33657930
cost: 63657930
x1: [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
x2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22151, 0, 0, 6565, 36796, 23350, 0, 0, 4579, 0, 0, 6559, 0, 0, 0, 0, 32897, 2175, 9722, 6485, 0, 0, 0, 12600, 11413, 0, 0, 0, 0, 8427, 7613, 4687, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [31]:
x1 = [solver.Value(x1[i]) for i in range(num_cities)]
x2 = [solver.Value(x2[i]) for i in range(num_cities)]
warehouses = np.array(np.concatenate((x1,x2)))
for i, val in enumerate(warehouses):
    if val > 0:
        print(cities[i % num_cities], val * i // num_cities + 1)

Lausanne 1
Lucerne 1
Zurich 1


In [34]:
sent = np.array([solver.Value(c[i][j]) for i in range(num_cities) for j in range(num_cities)])
for i, val in enumerate(sent):
    if val > 0:
        from_i = i // num_cities
        to_i = i% num_cities
        print('from:',cities[from_i],'to:',cities[to_i], 'amount:', val, 'distance:', distance[from_i][to_i],'km', 'price:', distance[from_i][to_i] * val)

from: Lausanne to: Bern amount: 22151 distance: 105 km price: 2325855
from: Lausanne to: Fribourg amount: 6565 distance: 72 km price: 472680
from: Lausanne to: Geneva amount: 36796 distance: 71 km price: 2612516
from: Lausanne to: Lausanne amount: 23350 distance: 0 km price: 0
from: Lausanne to: Montreux amount: 4579 distance: 29 km price: 132791
from: Lausanne to: Sion amount: 6559 distance: 95 km price: 623105
from: Lucerne to: Basel amount: 32897 distance: 95 km price: 3125215
from: Lucerne to: Bern amount: 2175 distance: 116 km price: 252300
from: Lucerne to: Biel amount: 9722 distance: 105 km price: 1020810
from: Lucerne to: Chur amount: 6485 distance: 143 km price: 927355
from: Lucerne to: Lucerne amount: 12600 distance: 0 km price: 0
from: Lucerne to: Lugano amount: 11413 distance: 171 km price: 1951623
from: Lucerne to: Thun amount: 8427 distance: 97 km price: 817419
from: Lucerne to: Winterthur amount: 7613 distance: 77 km price: 586201
from: Lucerne to: Zug amount: 4687 dista